In [1]:
library(lme4)
library(afex)
library(emmeans)
library(dplyr)
library(ggplot2)
library(svglite)
library(lmtest)
library(tidyr)
library(MCMCglmm)
library(reshape)
library(gridExtra)
library(cowplot)
library(svglite)
library(data.table)

Loading required package: Matrix



************
Welcome to afex. For support visit: http://afex.singmann.science/

- Functions for ANOVAs: aov_car(), aov_ez(), and aov_4()
- Methods for calculating p-values with mixed(): 'S', 'KR', 'LRT', and 'PB'
- 'afex_aov' and 'mixed' objects can be passed to emmeans() for follow-up tests
- Get and set global package options with: afex_options()
- Set sum-to-zero contrasts globally: set_sum_contrasts()
- For example analyses see: browseVignettes("afex")
************


Attaching package: ‘afex’


The following object is masked from ‘package:lme4’:

    lmer



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘tidyr’


The following objects are masked from ‘package:Matrix’:


In [2]:
ITI <- read.csv('/home/bagotlab/eshaan.i/PRL/FP_PRL_Nov_2021/CLEAN/Data/PRL_ZScore_ITI.csv')
ITI = subset(ITI,ID!=223)
ITI$time_idx <- as.factor(as.numeric(as.factor(ITI$time)))
ITI<-ITI[,c('index','time_idx','time','ID','rewards','lp','LeftLeverProb','Day','rt','PFC','vHIP')]
ITI$sex <- as.factor(ifelse(ITI$ID > 212,"Female", "Male"))
ITI$Day <- as.numeric(as.factor(ITI$Day))

ITI = subset(ITI, lp > 0)
ITI$time_idx <- as.factor(as.numeric(as.factor(ITI$time)))
ITI<- ITI %>% group_by(ID,Day) %>% mutate(Previous.Reward = lag(rewards,n=240))
ITI<- ITI %>% group_by(ID,Day) %>% mutate(Next.Reward = lead(rewards,n=240))
#created lagged dataframe where t2 is now t1
ITI2<-ITI[!is.na(ITI$Previous.Reward),]
ITI<-ITI %>% group_by(ID,Day) %>% mutate(DualTrialTime_Idx = rep_len(1:480, length.out=n()),DualTrial_Idx=rep(1:ceiling(n()/480), each=480, length.out=n()))
ITI2<-ITI2 %>% group_by(ID,Day) %>% mutate(DualTrialTime_Idx = rep_len(1:480, length.out=n()),DualTrial_Idx=rep(1:ceiling(n()/480), each=480, length.out=n()))

ITI = ITI %>% group_by(ID,Day,DualTrial_Idx) %>% mutate(last = DualTrialTime_Idx[[length(DualTrialTime_Idx)]])
ITI2 = ITI2 %>% group_by(ID,Day,DualTrial_Idx) %>% mutate(last = DualTrialTime_Idx[[length(DualTrialTime_Idx)]])

#remove orphan trials
ITI = subset(ITI, last==480)
ITI2 = subset(ITI2, last==480)

ITI$DF <- 0
ITI2$DF <- 1

ITI = rbind(ITI,ITI2)
ITI <- ITI %>% mutate(NewIdx = paste(Day,ID,DualTrial_Idx,DF))
ITI$NewIdx = as.numeric(as.factor(ITI$NewIdx))
ITI<-ITI %>% group_by(NewIdx) %>% mutate(CompoundReward = as.factor(paste(rewards[1],Next.Reward[1])), R1 = as.factor(rewards[1]), R2=as.factor(Next.Reward[1]))

prechoice1<- as.data.frame(ITI[((ITI$DualTrialTime_Idx)>=1)&((ITI$DualTrialTime_Idx)<=40),] %>% group_by(NewIdx)%>%summarize(PFC_Pre1=mean(PFC),vHIP_Pre1=mean(vHIP)))
ITI = merge(ITI,prechoice1,by = 'NewIdx')

#calculate with baseline centered
ITI$PFC = ITI$PFC - ITI$PFC_Pre1
ITI$vHIP = ITI$vHIP - ITI$vHIP_Pre1
prechoice1<- as.data.frame(ITI[((ITI$DualTrialTime_Idx)>=1)&((ITI$DualTrialTime_Idx)<=40),] %>% group_by(NewIdx)%>%summarize(PFC=mean(PFC),vHIP=mean(vHIP),R1=first(R1), R2=first(R2), ID=as.factor(mean(ID)),Day=as.factor(mean(Day)), sex=first(sex)))
enditi1<- as.data.frame(ITI[((ITI$DualTrialTime_Idx)>=200)&((ITI$DualTrialTime_Idx)<=240),] %>% group_by(NewIdx)%>%summarize(PFC=mean(PFC),vHIP=mean(vHIP),R1=first(R1), R2=first(R2), ID=as.factor(mean(ID)),Day=as.factor(mean(Day)), sex=first(sex)))
prechoice2<- as.data.frame(ITI[((ITI$DualTrialTime_Idx)>=241)&((ITI$DualTrialTime_Idx)<=280),] %>% group_by(NewIdx)%>%summarize(PFC=mean(PFC),vHIP=mean(vHIP),R1=first(R1), R2=first(R2), ID=as.factor(mean(ID)),Day=as.factor(mean(Day)), sex=first(sex)))
enditi2<- as.data.frame(ITI[((ITI$DualTrialTime_Idx)>=440)&((ITI$DualTrialTime_Idx)<=480),] %>% group_by(NewIdx)%>%summarize(PFC=mean(PFC),vHIP=mean(vHIP),R1=first(R1), R2=first(R2), ID=as.factor(mean(ID)),Day=as.factor(mean(Day)), sex=first(sex)))

In [3]:
plot_theme <- theme(axis.title=element_text(size=25,color='gray30'),
                    axis.line=element_line(color='gray30'),
                    axis.text.y=element_text(size=20,color='gray30'),
                    axis.text.x=element_text(size=20,color='gray30'),
                    legend.text=element_text(size=20),
                    legend.key.size=unit(2,'mm'),
                    panel.background=element_blank(),
                    panel.grid.major.y = element_blank(),
                    legend.position = 'none',
                    plot.margin = margin(0.5,0.5,0.5,0.5,'cm'),
                    strip.background = element_blank(),
                    strip.text = element_text(size=20,color='gray30')) 

plot_theme_PFC <- list(plot_theme, scale_color_manual(values=c('1.0'="#9E9E9E",'1.1'="#B51D1D",  '0.0'="#444444",'0.1'="#EC407A")),
                        scale_fill_manual(values=c('1.0'="#9E9E9E",'1.1'="#B51D1D",  '0.0'="#444444",'0.1'="#EC407A")),
                        scale_shape_manual(values=c('Male'=16,'Female'=17)))
plot_theme_vHIP <- list(plot_theme, scale_color_manual(values=c('1.0'="#9E9E9E" ,'1.1'="#F27E19", '0.0'="#444444",'0.1'="#FFC107")), 
                        scale_fill_manual(values=c('1.0'="#9E9E9E" ,'1.1'="#F27E19", '0.0'="#444444",'0.1'="#FFC107")),
                        scale_shape_manual(values=c('Male'=16,'Female'=17)))

In [ ]:
PFC1Back = NULL
for (tm in 1:480){
myModel=lmer(PFC ~ R1*R2*sex +(1|ID)+(1|Day),data=subset(ITI, (DualTrialTime_Idx==tm)))
current_time = (tm/20)-2
PFC1Back= rbind(PFC1Back,cbind(time=current_time,as.data.frame(emmeans(myModel, ~ R1*R2|sex))))
}

fig1<-ggplot(data = PFC1Back)+
  geom_line(aes(time,emmean,group=interaction(R1,R2), colour=interaction(R1,R2)))+
  geom_ribbon(aes(x=time,y = emmean, ymin = emmean - SE, ymax = emmean + SE, fill = interaction(R1,R2)), alpha = .2)+
  xlab('Time from LP')+
  ylab('mPFC Activity')+
  geom_vline(xintercept=c(0,10,12,22), linetype="dotted", size = 1)+
  plot_theme_PFC+
  facet_grid(~sex)

PFC_rewards_enditi <- lmer(PFC~R1*R2*sex+(1|Day)+(1|ID), REML=TRUE ,data = enditi1)
PFC_rewards_enditi_means <- emmeans(PFC_rewards_enditi, ~ R1*R2|sex)
dtf <- PFC_rewards_enditi_means
simp <- pairs(dtf, simple = "each")
test(simp, by = NULL)
Bulk <- enditi1 %>% group_by(ID, R1,R2, sex) %>% summarise(PFC=mean(PFC, na.rm = TRUE))
dtf<- data.frame(dtf)
fig2<-ggplot(dtf) +
  geom_point( aes(x=interaction(R1,R2), y=emmean, color=interaction(R1,R2)), stat="identity", size = 5, alpha=0.6) +
  geom_errorbar( aes(x=interaction(R1,R2), ymin=emmean-SE, ymax=emmean+SE,color=interaction(R1,R2)), width=0.2, alpha=0.6, size=1.3) +
  geom_jitter(data=Bulk, aes(x = interaction(R1,R2), y = PFC, color = interaction(R1,R2), shape=sex), size = 2, width = 0.1) +
  ylab('mPFC Activity')+
  plot_theme_PFC +
  facet_grid(~sex)

PFC_rewards_enditi <- lmer(PFC~R1*R2*sex+(1|Day)+(1|ID), REML=TRUE ,data = enditi2)
PFC_rewards_enditi_means <- emmeans(PFC_rewards_enditi, ~ R1*R2|sex)
dtf <- PFC_rewards_enditi_means
simp <- pairs(dtf, simple = "each")
test(simp, by = NULL)
Bulk <- enditi2 %>% group_by(ID, R1,R2,sex) %>% summarise(PFC=mean(PFC, na.rm = TRUE))
dtf<- data.frame(dtf)
fig3<-ggplot(dtf) +
  geom_point( aes(x=interaction(R1,R2), y=emmean, color=interaction(R1,R2)), stat="identity", size = 5, alpha=0.6) +
  geom_errorbar( aes(x=interaction(R1,R2), ymin=emmean-SE, ymax=emmean+SE,color=interaction(R1,R2)), width=0.2, alpha=0.6, size=1.3) +
  geom_jitter(data=Bulk, aes(x = interaction(R1,R2), y = PFC, color = interaction(R1,R2), shape=sex), size = 2, width = 0.1) +
  ylab('mPFC Activity')+
  plot_theme_PFC+
  facet_grid(~sex)

In [ ]:
vHIP1Back = NULL
for (tm in 1:480){
myModel=lmer(vHIP ~ R1*R2*sex +(1|ID)+(1|Day),data=subset(ITI, (DualTrialTime_Idx==tm)))
current_time = (tm/20)-2
vHIP1Back= rbind(vHIP1Back,cbind(time=current_time,as.data.frame(emmeans(myModel, ~ R1*R2|sex))))
}

fig4<-ggplot(data = vHIP1Back)+
  geom_line(aes(time,emmean,group=interaction(R1,R2), colour=interaction(R1,R2)))+
  geom_ribbon(aes(x=time,y = emmean, ymin = emmean - SE, ymax = emmean + SE, fill = interaction(R1,R2)), alpha = .2)+
  xlab('Time from LP')+
  ylab('vHip Activity')+
  geom_vline(xintercept=c(0,10,12,22), linetype="dotted", size = 1)+
  plot_theme_vHIP+
  facet_grid(~sex)

PFC_rewards_enditi <- lmer(vHIP~R1*R2*sex+(1|Day)+(1|ID), REML=TRUE ,data = enditi1)
PFC_rewards_enditi_means <- emmeans(PFC_rewards_enditi, ~ R1*R2|sex)
dtf <- PFC_rewards_enditi_means
simp <- pairs(dtf, simple = "each")
test(simp, by = NULL)
Bulk <- enditi1 %>% group_by(ID, R1,R2) %>% summarise(PFC=mean(PFC, na.rm = TRUE))
dtf<- data.frame(dtf)
fig5<-ggplot(dtf) +
  geom_point( aes(x=interaction(R1,R2), y=emmean, color=interaction(R1,R2)), stat="identity", size = 5, alpha=0.6) +
  geom_errorbar( aes(x=interaction(R1,R2), ymin=emmean-SE, ymax=emmean+SE,color=interaction(R1,R2)), width=0.2, alpha=0.6, size=1.3) +
  geom_jitter(data=Bulk, aes(x = interaction(R1,R2), y = PFC, color = interaction(R1,R2), shape=sex), size = 2, width = 0.1) +
  ylab('vHip Activity')+
  plot_theme_vHIP+
  facet_grid(~sex)

PFC_rewards_enditi <- lmer(vHIP~R1*R2*sex+(1|Day)+(1|ID), REML=TRUE ,data = enditi2)
PFC_rewards_enditi_means <- emmeans(PFC_rewards_enditi, ~ R1*R2|sex)
dtf <- PFC_rewards_enditi_means
simp <- pairs(dtf, simple = "each")
test(simp, by = NULL)
Bulk <- enditi2 %>% group_by(ID, R1,R2,sex) %>% summarise(PFC=mean(PFC, na.rm = TRUE))
dtf<- data.frame(dtf)
fig6<-ggplot(dtf) +
  geom_point( aes(x=interaction(R1,R2), y=emmean, color=interaction(R1,R2)), stat="identity", size = 5, alpha=0.6) +
  geom_errorbar( aes(x=interaction(R1,R2), ymin=emmean-SE, ymax=emmean+SE,color=interaction(R1,R2)), width=0.2, alpha=0.6, size=1.3) +
  geom_jitter(data=Bulk, aes(x = interaction(R1,R2), y = PFC, color = interaction(R1,R2), shape=sex), size = 2, width = 0.1) +
  ylab('vHip Activity')+
  plot_theme_vHIP+
  facet_grid(~sex)

In [4]:
#Figure A

PFC_rewards_enditi <- lmer(PFC~R1*R2*sex+(1|Day)+(1|ID), REML=TRUE ,data = enditi2)
PFC_rewards_enditi_means <- emmeans(PFC_rewards_enditi, ~ R1*R2|sex)
dtf <- PFC_rewards_enditi_means
simp <- pairs(dtf, simple = "each")
test(simp, by = NULL)
Bulk <- enditi2 %>% group_by(ID, R1,R2,sex) %>% summarise(PFC=mean(PFC, na.rm = TRUE))
dtf<- data.frame(dtf)
summary(PFC_rewards_enditi)
fig3<-ggplot(dtf) +
  geom_point( aes(x=interaction(R1,R2), y=emmean, color=interaction(R1,R2)), stat="identity", size = 5, alpha=0.6) +
  geom_errorbar( aes(x=interaction(R1,R2), ymin=emmean-SE, ymax=emmean+SE,color=interaction(R1,R2)), width=0.2, alpha=0.6, size=1.3) +
  geom_jitter(data=Bulk, aes(x = interaction(R1,R2), y = PFC, color = interaction(R1,R2), shape=sex), size = 2, width = 0.1) +
  ylab('mPFC Activity')+
  plot_theme_PFC+
  facet_grid(~sex)

Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 29197' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 29197)' or larger];
but be warned that this may result in large computation time and memory use.

Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 29197' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 29197)' or larger];
but be warned that this may result in large computation time and memory use.

Note: adjust = "tukey" was changed to "sidak"
because "tukey" is only appropriate for one set of pairwise comparisons

Note: adjust = "tukey" was changed to "sidak"
because "tukey" is only appropriate for one set of pairwise comparisons

Note: adjust = "tukey" was changed to "sidak"
because "tukey" is only appropriate for one set of pairwise comparisons



,contrast,R2,sex,estimate,SE,df,z.ratio,p.value
,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,R10 - R11,1,Female,0.0352125336,0.01271456,Inf,2.76946592,2.227086e-02
2,R10 - R11,0,Female,0.0892167909,0.01185682,Inf,7.52451029,2.118306e-13
3,R10 - R11,1,Male,-0.0006366372,0.01279099,Inf,-0.04977234,9.999975e-01
4,R10 - R11,0,Male,0.0651514071,0.01186133,Inf,5.49275558,1.582841e-07
,contrast,R1,sex,estimate,SE,df,z.ratio,p.value
,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,R21 - R20,0,Female,-0.2374902,0.01185694,Inf,-20.02963,0
2,R21 - R20,1,Female,-0.1834860,0.01271460,Inf,-14.43112,0
3,R21 - R20,0,Male,-0.2414583,0.01186435,Inf,-20.35158,0


`summarise()` has grouped output by 'ID', 'R1', 'R2'. You can override using
the `.groups` argument.


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: PFC ~ R1 * R2 * sex + (1 | Day) + (1 | ID)
   Data: enditi2

REML criterion at convergence: 44917.4

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-12.3078  -0.4042   0.0092   0.4133  16.5794 

Random effects:
 Groups   Name        Variance  Std.Dev.
 ID       (Intercept) 2.634e-04 0.016230
 Day      (Intercept) 9.397e-05 0.009694
 Residual             2.720e-01 0.521525
Number of obs: 29197, groups:  ID, 21; Day, 6

Fixed effects:
                  Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)     -1.238e-01  1.077e-02  5.515e+01 -11.493 2.92e-16 ***
R11             -3.521e-02  1.272e-02  2.903e+04  -2.769  0.00562 ** 
R20              2.375e-01  1.186e-02  2.901e+04  20.030  < 2e-16 ***
sexMale         -1.748e-02  1.415e-02  9.613e+01  -1.236  0.21961    
R11:R20         -5.400e-02  1.736e-02  2.918e+04  -3.110  0.00187 ** 
R11:sexMale      3.585e-02  1.

In [5]:
#Figure B

vHIP_rewards_enditi <- lmer(vHIP~R1*R2*sex+(1|Day)+(1|ID), REML=TRUE ,data = enditi2)
vHIP_rewards_enditi_means <- emmeans(vHIP_rewards_enditi, ~ R1*R2|sex)
dtf <- vHIP_rewards_enditi_means
simp <- pairs(dtf, simple = "each")
test(simp, by = NULL)
Bulk <- enditi2 %>% group_by(ID, R1,R2,sex) %>% summarise(vHIP=mean(vHIP, na.rm = TRUE))
dtf<- data.frame(dtf)
summary(vHIP_rewards_enditi)
fig6<-ggplot(dtf) +
  geom_point( aes(x=interaction(R1,R2), y=emmean, color=interaction(R1,R2)), stat="identity", size = 5, alpha=0.6) +
  geom_errorbar( aes(x=interaction(R1,R2), ymin=emmean-SE, ymax=emmean+SE,color=interaction(R1,R2)), width=0.2, alpha=0.6, size=1.3) +
  geom_jitter(data=Bulk, aes(x = interaction(R1,R2), y = PFC, color = interaction(R1,R2), shape=sex), size = 2, width = 0.1) +
  ylab('vHip Activity')+
  plot_theme_vHIP+
  facet_grid(~sex)

Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 30660' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 30660)' or larger];
but be warned that this may result in large computation time and memory use.

Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 30660' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 30660)' or larger];
but be warned that this may result in large computation time and memory use.

Note: adjust = "tukey" was changed to "sidak"
because "tukey" is only appropriate for one set of pairwise comparisons

Note: adjust = "tukey" was changed to "sidak"
because "tukey" is only appropriate for one set of pairwise comparisons

Note: adjust = "tukey" was changed to "sidak"
because "tukey" is only appropriate for one set of pairwise comparisons



,contrast,R2,sex,estimate,SE,df,z.ratio,p.value
,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,R10 - R11,1,Female,2.502782e-02,0.01214827,Inf,2.060196389,1.484564e-01
2,R10 - R11,0,Female,7.924072e-02,0.01132870,Inf,6.994688095,1.063416e-11
3,R10 - R11,1,Male,7.666726e-05,0.01162406,Inf,0.006595567,1.000000e+00
4,R10 - R11,0,Male,9.853096e-02,0.01081935,Inf,9.106920441,0.000000e+00
,contrast,R1,sex,estimate,SE,df,z.ratio,p.value
,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,R21 - R20,0,Female,-0.12734867,0.01132879,Inf,-11.241152,0.000000e+00
2,R21 - R20,1,Female,-0.07313577,0.01214826,Inf,-6.020268,6.965140e-09
3,R21 - R20,0,Male,-0.14591322,0.01082261,Inf,-13.482259,0.000000e+00


`summarise()` has grouped output by 'ID', 'R1', 'R2'. You can override using
the `.groups` argument.


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: vHIP ~ R1 * R2 * sex + (1 | Day) + (1 | ID)
   Data: enditi2

REML criterion at convergence: 44359.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-8.4487 -0.4554  0.0059  0.4630 15.3882 

Random effects:
 Groups   Name        Variance  Std.Dev.
 ID       (Intercept) 0.0002234 0.01495 
 Day      (Intercept) 0.0003153 0.01776 
 Residual             0.2481586 0.49816 
Number of obs: 30660, groups:  ID, 22; Day, 6

Fixed effects:
                  Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)     -6.781e-02  1.193e-02  2.433e+01  -5.682 7.12e-06 ***
R11             -2.503e-02  1.215e-02  3.064e+04  -2.060  0.03939 *  
R20              1.273e-01  1.133e-02  3.062e+04  11.241  < 2e-16 ***
sexMale          7.172e-05  1.307e-02  1.090e+02   0.005  0.99563    
R11:R20         -5.421e-02  1.659e-02  3.064e+04  -3.268  0.00108 ** 
R11:sexMale      2.495e-02  1.679e-02  